In [1]:
import cfbd
import pandas as pd
from cfbd import Configuration, ApiClient, PlaysApi
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np

In [2]:
cd C:\Users\Nolanator\Documents\College Football

C:\Users\Nolanator\Documents\College Football


In [3]:
team_dict = pd.read_csv("College_dict_Nolan.csv",header=None)
team_dict.columns=["full", "abbr"]
team_dict["full"] = team_dict["full"].str.upper().str.strip()
team_dict=dict(zip(team_dict["full"], team_dict["abbr"]))
team_correct_dict = {"SAN JOSÃ© STATE":"SAN JOSE STATE", "HAWAI'I":"HAWAII", "SAN JOSÉ STATE":"SAN JOSE STATE", "EASTERN MICHIGAN":"EAST MICHIGAN"}

In [4]:
import time
import pandas as pd
import os
from cfbd import Configuration, ApiClient, PlaysApi

api_key = 'EvAfgJ4sLdKNrQf9uDQFCFl+iZWPKE8N8l7vv2ENWmONMY/H7T8VafU6n/nctXat'

# Set up API configuration
configuration = Configuration()
configuration.api_key['Authorization'] = 'Bearer ' + api_key
api_client = ApiClient(configuration)
plays_api = PlaysApi(api_client)

# File to save progress
save_path = "cfb_all_plays_2024.csv"

# Load existing data if file exists
if os.path.exists(save_path):
    df_existing = pd.read_csv(save_path)
    downloaded_weeks = set(zip(df_existing["week"], df_existing["season_type"]))
    print(f"🔄 Resuming from existing data: {len(downloaded_weeks)} weeks already saved.")
else:
    df_existing = pd.DataFrame()
    downloaded_weeks = set()

all_plays = df_existing.to_dict("records")  # Convert existing data to list of dicts

def fetch_weekly_data(year, week, season_type="regular", max_retries=3):
    """Fetch play data for a given week, with retries and delay handling."""
    attempt = 0
    while attempt < max_retries:
        try:
            print(f"🔄 Fetching Week {week} ({season_type})...")
            plays = plays_api.get_plays(year=year, week=week, season_type=season_type)
            play_data = []

            for play in plays:
                play_data.append({
                    'week': week,
                    'season_type': season_type,
                    'away': play.away,
                    'clock_minutes': play.clock.minutes if play.clock else None,
                    'clock_seconds': play.clock.seconds if play.clock else None,
                    'defense': play.defense,
                    'defense_conference': play.defense_conference,
                    'defense_score': play.defense_score,
                    'defense_timeouts': play.defense_timeouts,
                    'distance': play.distance,
                    'down': play.down,
                    'drive_id': play.drive_id,
                    'drive_number': play.drive_number,
                    'game_id': play.game_id,
                    'home': play.home,
                    'id': play.id,
                    'offense': play.offense,
                    'offense_conference': play.offense_conference,
                    'offense_score': play.offense_score,
                    'offense_timeouts': play.offense_timeouts,
                    'period': play.period,
                    'play_number': play.play_number,
                    'play_text': play.play_text,
                    'play_type': play.play_type,
                    'ppa': play.ppa,
                    'scoring': play.scoring,
                    'wallclock': play.wallclock,
                    'yard_line': play.yard_line,
                    'yards_gained': play.yards_gained,
                    'yards_to_goal': play.yards_to_goal
                })

            print(f"✅ Week {week} ({season_type}) completed! {len(play_data)} plays fetched.")
            return play_data

        except Exception as e:
            print(f"⚠️ Error fetching Week {week} ({season_type}) (Attempt {attempt + 1}/{max_retries}): {e}")
            attempt += 1
            time.sleep(5)  # Wait before retrying

    print(f"❌ Skipping Week {week} ({season_type}) after {max_retries} failed attempts.")
    return []

# Fetch regular season (Weeks 1-16)
for week in range(1, 17):
    if (week, "regular") in downloaded_weeks:
        print(f"⏭️ Skipping Week {week} (regular), already downloaded.")
        continue

    weekly_data = fetch_weekly_data(year=2024, week=week, season_type="regular")
    all_plays.extend(weekly_data)

    # Save progress after each week
    pd.DataFrame(all_plays).to_csv(save_path, index=False)
    print(f"💾 Saved Week {week} (regular) progress.")

    time.sleep(2)  # Short delay to avoid API rate limits

# Fetch postseason (Weeks 1-4)
for week in range(1, 5):
    if (week, "postseason") in downloaded_weeks:
        print(f"⏭️ Skipping Week {week} (postseason), already downloaded.")
        continue

    weekly_data = fetch_weekly_data(year=2024, week=week, season_type="postseason")
    all_plays.extend(weekly_data)

    # Save progress after each week
    pd.DataFrame(all_plays).to_csv(save_path, index=False)
    print(f"💾 Saved Week {week} (postseason) progress.")

    time.sleep(2)  # Short delay to avoid API rate limits

print("🎉 All weeks fetched successfully! Data saved.")


🔄 Resuming from existing data: 17 weeks already saved.
⏭️ Skipping Week 1 (regular), already downloaded.
⏭️ Skipping Week 2 (regular), already downloaded.
⏭️ Skipping Week 3 (regular), already downloaded.
⏭️ Skipping Week 4 (regular), already downloaded.
⏭️ Skipping Week 5 (regular), already downloaded.
⏭️ Skipping Week 6 (regular), already downloaded.
⏭️ Skipping Week 7 (regular), already downloaded.
⏭️ Skipping Week 8 (regular), already downloaded.
⏭️ Skipping Week 9 (regular), already downloaded.
⏭️ Skipping Week 10 (regular), already downloaded.
⏭️ Skipping Week 11 (regular), already downloaded.
⏭️ Skipping Week 12 (regular), already downloaded.
⏭️ Skipping Week 13 (regular), already downloaded.
⏭️ Skipping Week 14 (regular), already downloaded.
⏭️ Skipping Week 15 (regular), already downloaded.
⏭️ Skipping Week 16 (regular), already downloaded.
⏭️ Skipping Week 1 (postseason), already downloaded.
🔄 Fetching Week 2 (postseason)...
✅ Week 2 (postseason) completed! 0 plays fetched.
💾

In [5]:
df=pd.DataFrame(all_plays)

In [18]:
tos_partial_int_weight_factor=0.36
defence_boosted_weight_factor=0.55
special_teams_weight_factor=0.15

In [19]:
df.head()

,week,season_type,away,clock_minutes,clock_seconds,defense,defense_conference,defense_score,defense_timeouts,distance,...,period,play_number,play_text,play_type,ppa,scoring,wallclock,yard_line,yards_gained,yards_to_goal
0,1,regular,Western Kentucky,15,0,Western Kentucky,Conference USA,0,3.0,10,...,1,1,Conor Talty kickoff for 63 yds for a touchback,Kickoff,NaN,False,2024-08-31T23:11:51.000Z,35,0,65
1,1,regular,Western Kentucky,15,0,Alabama,SEC,0,3.0,10,...,1,2,TJ Finley pass complete to Kisean Johnson for ...,Pass Reception,-0.452604,False,2024-08-31T23:11:51.000Z,75,2,75
2,1,regular,Western Kentucky,14,38,Alabama,SEC,0,3.0,8,...,1,3,TJ Finley pass complete to River Helms for 16 ...,Pass Reception,2.065042,False,2024-08-31T23:12:28.000Z,73,16,73
3,1,regular,Western Kentucky,14,20,Alabama,SEC,0,3.0,10,...,1,4,TJ Finley pass incomplete to River Helms,Pass Incompletion,-1.157148,False,2024-08-31T23:12:45.000Z,57,0,57
4,1,regular,Western Kentucky,14,5,Alabama,SEC,0,3.0,10,...,1,5,Elijah Young run for a loss of 2 yards to the ...,Rush,-1.098294,False,2024-08-31T23:13:31.000Z,57,-2,57


In [20]:
df.groupby(by="play_type").agg("mean")["ppa"].sort_values()

C:\Users\Nolanator\AppData\Local\Temp\ipykernel_8284\1963310263.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby(by="play_type").agg("mean")["ppa"].sort_values()


play_type
Blocked Field Goal Touchdown    -9.223592
Interception Return Touchdown   -8.466241
Fumble Return Touchdown         -8.385247
Blocked Punt Touchdown          -6.815622
Punt Return Touchdown           -6.791818
Fumble Recovery (Opponent)      -2.239105
Interception                    -2.090237
Pass Interception Return        -1.970661
Safety                          -1.906990
Sack                            -1.260270
Kickoff Return Touchdown        -1.014642
End of Regulation               -0.822123
placeholder                     -0.691862
Pass Incompletion               -0.680428
End Period                      -0.630274
Fumble Recovery (Own)           -0.546943
Rush                             0.035319
Pass Reception                   0.839591
Rushing Touchdown                2.537950
Passing Touchdown                3.262160
Blocked Field Goal                    NaN
Blocked Punt                          NaN
Defensive 2pt Conversion              NaN
End of Game             

In [26]:
df["play_type"].value_counts(normalize=True)

Rush                             0.361416
Pass Reception                   0.189434
Pass Incompletion                0.128051
Penalty                          0.050964
Punt                             0.049689
Timeout                          0.044238
Kickoff                          0.042015
Sack                             0.020557
Kickoff Return (Offense)         0.019376
Rushing Touchdown                0.018409
Passing Touchdown                0.017822
Field Goal Good                  0.012183
End Period                       0.011633
Pass Interception Return         0.005740
Fumble Recovery (Own)            0.005074
End of Half                      0.004784
End of Game                      0.004614
Field Goal Missed                0.004006
Fumble Recovery (Opponent)       0.003872
Interception                     0.002430
Interception Return Touchdown    0.000818
Uncategorized                    0.000605
Blocked Field Goal               0.000464
Fumble Return Touchdown          0

In [21]:
df["tos_partial_int_weight"] = np.where(df["play_type"].isin(["Fumble Recovery (Opponent)", "Fumble Return Touchdown"]),tos_partial_int_weight_factor,0)

df["tos_partial_int_weight"] = np.where(df["play_type"].isin(["Pass Interception Return", "Interception"]),tos_partial_int_weight_factor*0.75,0)

# df["defence_boosted_weight"] = np.where(df["play_type"].isin(["Fumble Recovery (Opponent)", "Fumble Return Touchdown","Pass Interception Return", "Interception"]),defence_boosted_weight_factor,defence_boosted_weight_factor*0.5)

df["defence_boosted_weight"] =0
df["special_teams_weight"] = np.where(df["play_type"].str.contains("Field Goal|Kickoff|Punt", na=False),special_teams_weight_factor,0)


In [22]:
df[df["play_type"].str.contains("Field Goal|Kickoff|Punt", na=False)]["ppa"].isnull().sum()/df[df["play_type"].str.contains("Field Goal|Kickoff|Punt", na=False)]["ppa"].shape[0]

0.9982809637875159

In [23]:
df["final_weight"] = (1-df["tos_partial_int_weight"]) * (1-df["defence_boosted_weight"]) * (1-df["special_teams_weight"])

In [25]:
df["final_weight"].value_counts(normalize=True)

1.00    0.863315
0.85    0.128514
0.73    0.008170
Name: final_weight, dtype: float64

In [30]:
np.mean(df["offense"]==df["home"])

0.5027615105190916

In [39]:
def adjust_ppa(row):
    if pd.isna(row['ppa']):  # Handle NaN values in 'ppa'
        return 0
    if row['offense'] == row['home']:
        return row['ppa']
    else:
        return -1*row['ppa']

In [40]:
df['adjusted_ppa'] = df.apply(adjust_ppa, axis=1)
df['adjusted_ppa_weighted'] = df['adjusted_ppa']*df["final_weight"]

In [41]:
df["final_weight"].value_counts(normalize=True)

1.00    0.863315
0.85    0.128514
0.73    0.008170
Name: final_weight, dtype: float64

In [53]:
def trimean(arr):
    q1, q3 = np.percentile(arr, [25, 75])
    median = np.median(arr)
    return (q1 + 2*median + q3) / 4

def truncated_mean(arr, percentile):
    lower, upper = np.percentile(arr, [100-percentile, percentile])
    truncated = arr[(arr >= lower) & (arr <= upper)]
    return np.mean(truncated)

# Group by game_id and calculate metrics
# Assuming df is your DataFrame and you've already applied the adjust_ppa function

# Modify the grouped aggregation to include non-null PPA count
grouped = df.groupby('game_id').agg('sum')
    


# Reset index to make game_id a column
grouped.reset_index(inplace=True)


C:\Users\Nolanator\AppData\Local\Temp\ipykernel_8284\2659622774.py:15: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  grouped = df.groupby('game_id').agg('sum')


In [35]:

# First, perform linear regression using scipy.stats.linregress
slope, intercept, r_value, p_value, std_err = stats.linregress(df_investigation['plays_trimean_product'], df_investigation['home_margin'])

# Plot the linear regression using seaborn
sns.regplot(x='plays_trimean_product', y='home_margin', data=df_investigation, ci=None, line_kws={'label':f"y={slope:.2f}x+{intercept:.2f}"})

# Add legend to show the linear regression equation
plt.legend()

# Add text to the plot to show the R-squared value and p-value
plt.text(x=df_investigation['plays_trimean_product'].min(), 
         y=df_investigation['home_margin'].max(), 
         s=f"R-squared = {r_value**2:.2f}\nP-value = {p_value:.3f}",
         verticalalignment='top', horizontalalignment='left')

plt.xlabel('Plays Trimean Product')
plt.ylabel('Home Margin')
plt.title('Linear Regression between Plays Trimean Product and Home Margin')

# Show plot
plt.show()



NameError: name 'df_investigation' is not defined

In [22]:
grouped['predicted_home_margin'] = grouped['adjusted_ppa'] + 1.5

In [14]:
grouped[["game_id", "predicted_home_margin"]].to_csv("C:/Users/Nolanator/Documents/College Football/epa_margins_new.csv")

In [58]:
(grouped["adjusted_ppa"] - grouped["adjusted_ppa_weighted"]).max()

5.575704671334485

In [57]:
grouped[["game_id", "predicted_home_margin"]]

,game_id,week,clock_minutes,clock_seconds,defense_score,defense_timeouts,distance,down,drive_id,drive_number,...,scoring,yard_line,yards_gained,yards_to_goal,tos_partial_int_weight,defence_boosted_weight,special_teams_weight,final_weight,adjusted_ppa,adjusted_ppa_weighted
0,401628319,172,1181,4818,2499,433.0,1418,355,5317558946277,2717,...,9,9260,949,9366,0.54,0,3.60,167.86,61.075613,59.998701
1,401628320,168,1034,4068,2947,114.0,1415,278,4578562849972,1972,...,10,9551,821,9133,0.00,0,3.15,164.85,61.944180,61.944180
2,401628321,158,985,4563,4275,404.0,1213,289,4827572420778,2358,...,12,8447,1142,8629,0.00,0,4.20,153.80,52.933379,51.910770
3,401628322,158,1035,4489,2265,440.0,1339,298,3996201805821,1921,...,10,7485,1114,7763,0.81,0,3.45,153.74,-31.898803,-30.056833
4,401628323,154,1022,3908,1034,393.0,1340,311,3871697035453,1733,...,7,8521,818,8477,0.27,0,3.45,150.28,29.530730,28.898664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1599,401729783,2688,1053,4590,2786,422.0,1374,300,4724342250256,2176,...,8,9113,889,7745,1.62,0,2.25,164.13,-0.278224,-2.276498
1600,401729785,2512,1053,4269,1871,437.0,1442,319,4354750871547,2147,...,6,8608,800,8860,0.81,0,3.15,153.04,20.573846,20.794406
1601,401729786,167,1045,4555,2261,113.0,1612,297,3744121607266,1746,...,7,9031,771,9467,0.00,0,2.70,164.30,0.361585,0.361585
1602,401729787,153,905,3906,2660,453.0,1325,298,3326322638001,1641,...,8,7849,892,8185,0.00,0,3.15,149.85,11.812165,11.812165


In [ ]:
grouped.tail()

In [24]:
df_investigation=pd.merge(left=grouped, right=df_game_scores ,how='left', on='game_id')

NameError: name 'df_game_scores' is not defined

In [ ]:
22.793628/132

In [ ]:
(0.36186291209454839573+0.07624720613379611712)/2

In [ ]:
grouped.to_csv("cfb_epa_game_scores.csv")

In [ ]:
df_investigation.shape

In [ ]:
grouped.plot(x="plays_median_product", y="home_margin", kind='scatter')

In [ ]:
df_investigation.plot(x="plays_trimean_product", y="home_margin", kind='scatter')

In [ ]:
df_investigation.plot(x="plays_mean_product", y="home_margin", kind='scatter')

In [ ]:
df_investigation.plot(x="plays_truncated_90_product", y="home_margin", kind='scatter')

In [13]:
grouped

,game_id,week,clock_minutes,clock_seconds,defense_score,defense_timeouts,distance,down,drive_id,drive_number,...,offense_timeouts,period,play_number,ppa,scoring,yard_line,yards_gained,yards_to_goal,adjusted_ppa,predicted_home_margin
0,401628319,172,1181,4818,2499,433.0,1418,355,5317558946277,2717,...,469.0,405,881,26.194711,9,9260,949,9366,61.075613,62.575613
1,401628320,168,1034,4068,2947,114.0,1415,278,4578562849972,1972,...,86.0,381,887,48.543855,10,9551,821,9133,61.944180,63.444180
2,401628321,158,985,4563,4275,404.0,1213,289,4827572420778,2358,...,379.0,354,664,25.009868,12,8447,1142,8629,52.933379,54.433379
3,401628322,158,1035,4489,2265,440.0,1339,298,3996201805821,1921,...,423.0,391,697,26.039802,10,7485,1114,7763,-31.898803,-30.398803
4,401628323,154,1022,3908,1034,393.0,1340,311,3871697035453,1733,...,420.0,390,739,16.276824,7,8521,818,8477,29.530730,31.030730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1599,401729783,2688,1053,4590,2786,422.0,1374,300,4724342250256,2176,...,423.0,404,879,19.465465,8,9113,889,7745,-0.278224,1.221776
1600,401729785,2512,1053,4269,1871,437.0,1442,319,4354750871547,2147,...,444.0,379,666,6.809946,6,8608,800,8860,20.573846,22.073846
1601,401729786,167,1045,4555,2261,113.0,1612,297,3744121607266,1746,...,261.0,426,931,30.902501,7,9031,771,9467,0.361585,1.861585
1602,401729787,153,905,3906,2660,453.0,1325,298,3326322638001,1641,...,439.0,374,778,22.268639,8,7849,892,8185,11.812165,13.312165


In [ ]:
df_investigation['predicted_home_margin']

In [ ]:
df_investigation[(df_investigation["home_team"]=="Penn State")|(df_investigation["away_team"]=="Penn State")]